In [ ]:
import requests
from IPython.display import clear_output 

import io, os, sys, ssl, time, json, socket, struct, datetime, threading
import requests, serial, repl, random #, picamera, pigpio
from time import gmtime, strftime, sleep

server_ip = 'yzlab3.chem.nyu.edu'#'guoxiaokang.net' 
flask_port = 54323
basic_auth_username = 'Hal?ou'
basic_auth_password = 'Wode*%^!()'

def chksum_nmea(sentence): 
    csum = 0  
    for c in sentence: 
        csum ^= ord(c)
    return str(hex(csum)).upper()[2:] 

def newGPRMC():  
    lat, lon = 4043.831, 7359.70
    for i in range(36,1000):
        lat = lat + 0.003*random.randint(-50,20) 
        lon = lon + 0.003*random.randint(-20,100) 
        timestamp = strftime("%H%M%S", gmtime())
        chksum = chksum_nmea('GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W'.format(timestamp,lat,lon))  
        yield '$GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W*{}'.format(timestamp,lat,lon,chksum)

In [ ]:
def dbd():
    for GPRMC in newGPRMC():  
        data = {}
        data['P'] = GPRMC 
        #data['N'] = int(microbit.send_command('print(compass.heading())') ) 
        #data['a'] = microbit.send_command("print(accelerometer.get_values())") 
        requests.post(f'https://{server_ip}:{flask_port}/sensor', 
                      data=json.dumps(data), 
                      auth=(basic_auth_username, basic_auth_password)) 
        sleep(3)
        #print(data)
dbd()    
#threading.Thread(target=dbd).start() 